#Predict mosquitos dayswise (groupby date -.-)
because if we don't, we have data points that are exactly the same except that the numMosquitos is different, which HAS to fuck up the classifier.

## load and prepare data

In [282]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn import cross_validation, preprocessing, metrics

In [283]:
df_train = pd.read_csv('aggregated_train_with_nummosquitos.csv')# from make_aggregated_with_Nummosquitos
#df_test = pd.read_csv('../test_filled_new.csv')
#orig = pd.read_csv('../test_filled_new.csv')
df_test = pd.read_csv('../test_filled_new.csv')

### encode species and trap, set unspecified to mix, drop strings

In [284]:
df_train['counter'] = 1

In [285]:
X = df_train.copy()


In [286]:
string_features = ['Address','Street','AddressNumberAndStreet','CodeSum']
species_encoder = preprocessing.LabelEncoder()
trap_encoder = preprocessing.LabelEncoder()
X_dump = X[string_features]
df_dump = df_test[string_features]
#X_dump = X[string_features]

df_test.Species[df_test.Species == 'UNSPECIFIED CULEX'] = 'CULEX PIPIENS/RESTUANS'
#X.Species = species_encoder.fit_transform(X.Species)
y = df_train['NumMosquitos']
#df_test.Species = species_encoder.transform(df_test.Species)
#df_train.Species = species_encoder.transform(df_train.Species)
X = X.drop(string_features,axis=1)

/home/thomas/anaconda3/lib/python3.4/site-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### groupby date, trap, species

either:

1. disregard species and only predict num mosquitos independently

2. predict number mosquitos per species (make new variable for each species or it won't fit in the dates!). In this case we also need to store information about in how many '50 sets' there was Wnvpresent.

In [287]:
groups = ['Date','Trap','Species']

In [288]:
tmp = X.groupby(groups)['NumMosquitos'].sum()


In [306]:
X.Species = species_encoder.fit_transform(X.Species)

In [289]:
X_tmp = X.join(X.groupby(groups)['NumMosquitos'].sum(), on= groups,rsuffix='_counter')

In [290]:
X_tmp.NumMosquitos_counter.max()

2532

In [291]:
y = X_tmp.NumMosquitos_counter
y.max()

2532

now NumMosquiots_counter actually represents the number of mosquitos caught at a datapoint, so we can regress over it.

In [292]:
from sklearn.cross_validation import train_test_split, ShuffleSplit,cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, SGDRegressor
import numpy as np

In [293]:
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn import cross_validation, preprocessing, metrics
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA

In [294]:

clf_list= [#RandomForestRegressor(n_estimators=100,n_jobs=-1),
           #RandomForestRegressor(n_estimators=400,n_jobs=-1),
           #RandomForestRegressor(n_estimators=1000,n_jobs=-1), 
           #Ridge(alpha = .01,normalize=True),
           #Ridge(alpha = .1,normalize=True),
           #Ridge(alpha = .1),
           #GradientBoostingRegressor(n_estimators = 100,max_depth=7),
          # GradientBoostingRegressor(n_estimators = 1000,max_depth=10),
          # GradientBoostingRegressor(n_estimators = 1000,max_depth=15,subsample = .7),#so far the best one, n_es = 400
           GradientBoostingRegressor(n_estimators = 1000,max_depth=15,learning_rate = 0.05),
          # GradientBoostingRegressor(n_estimators = 1000,max_depth=20,subsample = .7)
           #KNeighborsRegressor(),
           #SGDRegressor()
           #GradientBoostingRegressor(n_estimators = 100,max_depth=4),
           #GradientBoostingRegressor(n_estimators = 100,max_depth=2)
      ]
random_state= 46


In [295]:
del X_tmp['Trap']
del X_tmp['NumMosquitos']

In [296]:
y = X_tmp.NumMosquitos_counter

string_features = ['Date','Address','Street','AddressNumberAndStreet','CodeSum','Trap']

X = X.drop(string_features,axis=1)
df_test.Species[df_test.Species == 'UNSPECIFIED CULEX'] = 'CULEX PIPIENS/RESTUANS'
X.Species = species_encoder.fit_transform(X.Species)
#y = df_train['NumMosquitos']
df_test.Species = species_encoder.transform(df_test.Species)
df_train.Species = species_encoder.transform(df_train.Species)

In [309]:
X = X_tmp
X.columns
X.dtypes

Unnamed: 0                   int64
Date                        object
Species                      int64
Block                        int64
Latitude                   float64
Longitude                  float64
AddressAccuracy              int64
WnvPresent                   int64
Station                      int64
Tmax                         int64
Tmin                         int64
Tavg                       float64
Depart                     float64
DewPoint                     int64
WetBulb                    float64
Heat                         int64
Cool                         int64
Sunrise                    float64
Sunset                     float64
Depth                      float64
SnowFall                   float64
PrecipTotal                float64
StnPressure                float64
SeaLevel                   float64
ResultSpeed                float64
ResultDir                    int64
AvgSpeed                   float64
Year                         int64
Month               

In [299]:
y.max()

2532

In [308]:
X = scale(X)


err = []
r2 = []
#get lengths of y_arr -.-

X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=.33, random_state= random_state)

y_arr = pd.DataFrame(y_test)
i = 1
#clf = clf_list[1]
for clf in clf_list:
#for i in np.random.randint(1,size=4):
    i += 1
    X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=.33, random_state= random_state)
    clf.fit(X_train,y_train)
    yhat = clf.predict(X_test)
    err.append( mean_squared_error(yhat,y_test))
    r2.append(r2_score(yhat,y_test))
    y_arr[i] = yhat
    

/home/thomas/anaconda3/lib/python3.4/site-packages/sklearn/utils/validation.py:332: UserWarning: The scale function assumes floating point values as input, got object
  "got %s" % (estimator, X.dtype))


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [281]:
y_arr_mean = y_arr.drop(0,axis=1)
y_arr_mean = y_arr_mean.mean(axis=1)
y_arr_mean
r2_score(y_arr_mean,y_arr[0])

0.94163084338374337

In [200]:
r2

[0.94188315337272766,
 0.93642041714111368,
 0.94264323435029096,
 0.93137696339511422]

In [201]:
yhat.mean(),y_test.mean()

(101.30464780498828, 100.67580040380733)

In [202]:
err

[6981.9641858704917,
 7547.6122920137095,
 6946.8602973212483,
 8177.4240437923063]

In [203]:
sum((yhat-y_test)**2**1/2)

14175564.579913994

In [204]:
bla = pd.DataFrame(yhat,columns = ['hat'])
bla['orig'] = y_test

In [205]:
bla

,hat,orig
0,0.918003,1
1,21.745852,24
2,1.081652,1
3,54.619309,50
4,1.768874,1
5,317.736029,329
6,1.908213,2
7,16.730586,25
8,7.138885,8
9,43.003945,23


In [206]:
r3 = r2

In [311]:
clf.estimators_

array([[ DecisionTreeRegressor(compute_importances=None,
           criterion=<sklearn.tree._tree.FriedmanMSE object at 0x7fb695340030>,
           max_depth=20, max_features=None, max_leaf_nodes=None,
           min_density=None, min_samples_leaf=1, min_samples_split=2,
           random_state=<mtrand.RandomState object at 0x7fb6c72f7cf8>,
           splitter=<sklearn.tree._tree.PresortBestSplitter object at 0x7fb694a0d448>)],
       [ DecisionTreeRegressor(compute_importances=None,
           criterion=<sklearn.tree._tree.FriedmanMSE object at 0x7fb695340030>,
           max_depth=20, max_features=None, max_leaf_nodes=None,
           min_density=None, min_samples_leaf=1, min_samples_split=2,
           random_state=<mtrand.RandomState object at 0x7fb6c72f7cf8>,
           splitter=<sklearn.tree._tree.PresortBestSplitter object at 0x7fb694a0d448>)],
       [ DecisionTreeRegressor(compute_importances=None,
           criterion=<sklearn.tree._tree.FriedmanMSE object at 0x7fb695340030>,